In [48]:
# Step 1: Install Kaggle API
!pip install -q kaggle

# Step 2: Create kaggle.json with your credentials
import json
import os

kaggle_token = {
    "username": "himanshusoni001",
    "key": "7888d0ba07df9b4b51271fe3c97fac80"
}

# Write the token to the correct location
os.makedirs('~/.kaggle', exist_ok=True)
with open('~/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_token, f)

# Set permission
!chmod 600 ~/.kaggle/kaggle.json

In [49]:
# Step 3: Download dataset using Kaggle API
!kaggle competitions download -c house-prices-advanced-regression-techniques

# Step 4: Unzip the downloaded dataset
!unzip -o house-prices-advanced-regression-techniques.zip

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [50]:
# Step 5: Load and use the dataset
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import gridplot, column, row
from bokeh.palettes import Viridis256, Category10
from bokeh.transform import linear_cmap
from scipy.stats import gaussian_kde
import warnings
warnings.filterwarnings('ignore')

# Enable Bokeh output in notebook
output_notebook()

data = pd.read_csv('train.csv')

# Optional: Preview data
data.head()

Loading BokehJS ...

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [51]:
data.tail(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1450,1451,90,RL,60.0,9000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2009,WD,Normal,136000
1451,1452,20,RL,78.0,9262,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,New,Partial,287090
1452,1453,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
1453,1454,20,RL,90.0,17217,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Abnorml,84500
1454,1455,20,FV,62.0,7500,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2009,WD,Normal,185000
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


#                    **DATA AUDIT AND AVAILABILITY CHECK**

In [52]:
# 1. Shape of the dataset
print("\nShape of dataset:", data.shape)



Shape of dataset: (1460, 81)


In [53]:
# 2. Data types of each column
print("\nData types of each column:")
print(data.dtypes)


Data types of each column:
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object


In [54]:
# 3. Missing values per column
print("\nMissing values per column:")
print(data.isnull().sum())


Missing values per column:
Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [55]:
# 4. Data Information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [56]:
# 5. Check for negative or invalid values
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
print(numeric_cols)
print("\nNegative values:")
for col in numeric_cols:
    if (data[col] < 0).any():
        print(f"\033[91mColumn '{col}' has negative values.\033[0m")
        print(data[data[col] < 0][[col]])
    else:
        print(f"\033[92mColumn '{col}' has no negative values.\033[0m")

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

Negative values:
Column 'Id' has no negative values.
Column 'MSSubClass' has no negative values.
Column 'LotFrontage' has no negative values.
Column 'LotArea' has no negative values.
Column 'OverallQual' has no negative values.
Column 'OverallCond' has no negative values.
Column 'YearBuilt' has no negative values.
Column 'YearRemodAdd' has no negative values.
Column 'MasVnrArea' has no negative values.
Column 'BsmtFinSF1' has no negative values.
Column 'BsmtFinSF2' has no ne

In [57]:
# 6. Duplicate rows
print("\nNumber of duplicate rows:", data.duplicated().sum())


Number of duplicate rows: 0


# **EXPLORATORY DATA ANALYSIS**

In [58]:
#Summary statistics for numeric columns
print("\nSummary statistics:")
print(data.describe())


Summary statistics:
                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.06

In [59]:
#Summary statistics for categorical columns
for col in data.columns:
    if (data[col].dtype=='object'):
        print(f"\n{col}:")
        print(data[col].value_counts())
    else:
        pass


MSZoning:
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64

Street:
Street
Pave    1454
Grvl       6
Name: count, dtype: int64

Alley:
Alley
Grvl    50
Pave    41
Name: count, dtype: int64

LotShape:
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: count, dtype: int64

LandContour:
LandContour
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: count, dtype: int64

Utilities:
Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64

LotConfig:
LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: count, dtype: int64

LandSlope:
LandSlope
Gtl    1382
Mod      65
Sev      13
Name: count, dtype: int64

Neighborhood:
Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR     

In [60]:
# Calculate average property prices across neighborhoods
average_price_by_neighborhood = data.groupby('Neighborhood')['SalePrice'].mean().sort_values()

# Identify the most affordable neighborhoods
affordable_neighborhoods = average_price_by_neighborhood.head(5)

# Identify the most expensive neighborhoods
expensive_neighborhoods = average_price_by_neighborhood.tail(5)

# Average property size and price by building type
average_size_and_price = data.groupby('BldgType')[['GrLivArea', 'SalePrice']].mean()

# Average price by overall quality
average_price_by_quality = data.groupby('OverallQual')['SalePrice'].mean().sort_values()

print("Average Price by Neighborhood:")
print(average_price_by_neighborhood)

print("\nMost Affordable Neighborhoods:")
print(affordable_neighborhoods)

print("\nMost Expensive Neighborhoods:")
print(expensive_neighborhoods)

print("\nAverage Living Area and Price by Building Type:")
print(average_size_and_price)

print("\nAverage Price by Overall Quality:")
print(average_price_by_quality)

Average Price by Neighborhood:
Neighborhood
MeadowV     98576.470588
IDOTRR     100123.783784
BrDale     104493.750000
BrkSide    124834.051724
Edwards    128219.700000
OldTown    128225.300885
Sawyer     136793.135135
Blueste    137500.000000
SWISU      142591.360000
NPkVill    142694.444444
NAmes      145847.080000
Mitchel    156270.122449
SawyerW    186555.796610
NWAmes     189050.068493
Gilbert    192854.506329
Blmngtn    194870.882353
CollgCr    197965.773333
Crawfor    210624.725490
ClearCr    212565.428571
Somerst    225379.837209
Veenker    238772.727273
Timber     242247.447368
StoneBr    310499.000000
NridgHt    316270.623377
NoRidge    335295.317073
Name: SalePrice, dtype: float64

Most Affordable Neighborhoods:
Neighborhood
MeadowV     98576.470588
IDOTRR     100123.783784
BrDale     104493.750000
BrkSide    124834.051724
Edwards    128219.700000
Name: SalePrice, dtype: float64

Most Expensive Neighborhoods:
Neighborhood
Veenker    238772.727273
Timber     242247.447368
Sto

In [61]:
# UNIVARIATE VISUALISATION
for col in numeric_cols:
    # Prepare data
    col_data = data[col].dropna()
    hist, edges = np.histogram(col_data, bins=20)

    # Create KDE
    kde = gaussian_kde(col_data)
    x_kde = np.linspace(col_data.min(), col_data.max(), 200)
    y_kde = kde(x_kde)
    # Normalize KDE to match histogram scale
    y_kde = y_kde * len(col_data) * (edges[1] - edges[0])

    # Create Bokeh figure
    p = figure(width=600, height=400, title=f'Histogram & KDE of {col}',
               x_axis_label=col, y_axis_label='Frequency')

    # Add histogram
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color='skyblue', line_color='white', alpha=0.7)

    # Add KDE line
    p.line(x_kde, y_kde, line_width=2, line_color='navy', legend_label='KDE')

    show(p)

In [62]:
# Correlation heat map
from bokeh.models import ColorBar, LinearColorMapper
from bokeh.transform import transform

numeric_data = data.select_dtypes(include=[np.number])
corr_matrix = numeric_data.corr()
corr_df = corr_matrix.stack().reset_index()
corr_df.columns = ['Feature1', 'Feature2', 'Correlation']

# Create color mapper
mapper = LinearColorMapper(palette=Viridis256,
                           low=corr_df['Correlation'].min(),
                           high=corr_df['Correlation'].max())

# Create figure for full correlation matrix
features = list(corr_matrix.columns)
p1 = figure(width=800, height=600, title='Correlation Heatmap',
           x_range=features, y_range=list(reversed(features)),
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

# Prepare data for heatmap - use proper indexing
source = ColumnDataSource(corr_df)

p1.rect('Feature1', 'Feature2', 1, 1, source=source, line_color=None,
        fill_color=transform('Correlation', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
p1.add_layout(color_bar, 'right')

p1.xaxis.major_label_orientation = 1.2
show(p1)

# --- Correlation Matrix With target---
num_cols = numeric_data.columns
target = 'SalePrice'
corr = data[num_cols].corr()
corr_target = corr[target].sort_values(ascending=False).to_frame()
corr_target.columns = ['Correlation']

# Create figure for target correlation
p2 = figure(width=600, height=800, title="Correlation of Features with SalePrice",
           x_range=list(corr_target.index), y_axis_label='Correlation',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

# Create color mapper for target correlation
mapper2 = LinearColorMapper(palette=Viridis256,
                            low=corr_target['Correlation'].min(),
                            high=corr_target['Correlation'].max())

# Format correlation values as strings for text labels
corr_text = [f'{val:.2f}' for val in corr_target['Correlation'].values]

source2 = ColumnDataSource(dict(
    feature=corr_target.index,
    correlation=corr_target['Correlation'].values,
    corr_text=corr_text
))

p2.rect(x='feature', y=0, width=0.8, height='correlation', source=source2,
        fill_color=transform('correlation', mapper2), line_color='white')

p2.xaxis.major_label_orientation = 1.2
p2.yaxis.axis_label = 'Correlation with SalePrice'

# Add text labels for correlation values
p2.text(x='feature', y='correlation', text='corr_text',
        source=source2, text_align='center', text_baseline='middle',
        text_font_size='8pt')

show(p2)

In [63]:
# --- Target Distribution (SalePrice) ---

# Before log transform
target_data = data[target].dropna()
hist_before, edges_before = np.histogram(target_data, bins=40)
kde_before = gaussian_kde(target_data)
x_kde_before = np.linspace(target_data.min(), target_data.max(), 200)
y_kde_before = kde_before(x_kde_before)
y_kde_before = y_kde_before * len(target_data) * (edges_before[1] - edges_before[0])

p1 = figure(width=800, height=400, title="SalePrice Distribution (Before Log Transform)",
           x_axis_label="SalePrice", y_axis_label='Frequency')
p1.quad(top=hist_before, bottom=0, left=edges_before[:-1], right=edges_before[1:],
        fill_color='teal', line_color='white', alpha=0.7)
p1.line(x_kde_before, y_kde_before, line_width=2, line_color='darkblue', legend_label='KDE')
show(p1)

# Check skewness and log-transform if needed
skew_val = data[target].skew()
print(f"Skewness of {target}: {skew_val:.2f}")
if skew_val > 0.75:
    data['SalePrice_log'] = np.log1p(data[target])
    # After log transform
    log_data = data['SalePrice_log'].dropna()
    hist_after, edges_after = np.histogram(log_data, bins=40)
    kde_after = gaussian_kde(log_data)
    x_kde_after = np.linspace(log_data.min(), log_data.max(), 200)
    y_kde_after = kde_after(x_kde_after)
    y_kde_after = y_kde_after * len(log_data) * (edges_after[1] - edges_after[0])

    p2 = figure(width=800, height=400, title="SalePrice Distribution (After Log Transform)",
               x_axis_label="Log(SalePrice)", y_axis_label='Frequency')
    p2.quad(top=hist_after, bottom=0, left=edges_after[:-1], right=edges_after[1:],
            fill_color='orange', line_color='white', alpha=0.7)
    p2.line(x_kde_after, y_kde_after, line_width=2, line_color='darkorange', legend_label='KDE')
    show(p2)


Skewness of SalePrice: 1.88


In [64]:
# 1. Log Transformation Visualization - Before and After Comparison
from bokeh.plotting import figure, show
import numpy as np

# Prepare data
target_original = data['SalePrice'].dropna()
target_log = data['SalePrice_log'].dropna() if 'SalePrice_log' in data.columns else np.log1p(target_original)

# Create side-by-side comparison
p1 = figure(width=400, height=400, title="Original SalePrice Distribution",
           x_axis_label="SalePrice", y_axis_label="Frequency",
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

p2 = figure(width=400, height=400, title="Log-Transformed SalePrice Distribution",
           x_axis_label="Log(SalePrice)", y_axis_label="Frequency",
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

# Histogram for original
hist1, edges1 = np.histogram(target_original, bins=40)
p1.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
        fill_color='steelblue', line_color='white', alpha=0.7)

# Histogram for log-transformed
hist2, edges2 = np.histogram(target_log, bins=40)
p2.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:],
        fill_color='coral', line_color='white', alpha=0.7)

# Show plots side by side
from bokeh.layouts import row
show(row(p1, p2))


In [65]:
#BIVARIATE VISUALISATION

# Average price by Neighborhood
# Compute mean sale price per neighborhood and sort
sorted_data = data.groupby('Neighborhood')['SalePrice'].mean().sort_values().reset_index()

p = figure(width=900, height=400, title='Average Sale Price by Neighborhood',
           x_range=sorted_data['Neighborhood'].tolist(),
           x_axis_label='Neighborhood', y_axis_label='Sale Price',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source = ColumnDataSource(sorted_data)
p.circle(x='Neighborhood', y='SalePrice', source=source, size=8, color='steelblue')
p.line(x='Neighborhood', y='SalePrice', source=source, line_width=2, color='steelblue')

p.xaxis.major_label_orientation = 1.2
show(p)

In [66]:
# Average price by Overall Quality
avg_by_qual = data.groupby('OverallQual')['SalePrice'].mean().reset_index()
avg_by_qual['OverallQual'] = avg_by_qual['OverallQual'].astype(str)

p = figure(width=700, height=400, title='Average Sale Price by Overall Quality',
           x_range=avg_by_qual['OverallQual'].tolist(),
           x_axis_label='Overall Quality', y_axis_label='Sale Price',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source = ColumnDataSource(avg_by_qual)
p.circle(x='OverallQual', y='SalePrice', source=source, size=10, color='coral')
p.line(x='OverallQual', y='SalePrice', source=source, line_width=2, color='coral')
show(p)

In [67]:
# Average price by Building Type
avg_by_bldg = data.groupby('BldgType')['SalePrice'].mean().reset_index()

p = figure(width=700, height=400, title='Average Sale Price by Building Type',
           x_range=avg_by_bldg['BldgType'].tolist(),
           x_axis_label='Building Type', y_axis_label='Sale Price',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source = ColumnDataSource(avg_by_bldg)
p.circle(x='BldgType', y='SalePrice', source=source, size=10, color='mediumseagreen')
p.line(x='BldgType', y='SalePrice', source=source, line_width=2, color='mediumseagreen')
show(p)

# **DATA CLEANING**

In [68]:
# Handle missing values
# First, let's identify columns with high missing percentages (>50%)
missing_pct = (data.isnull().sum() / len(data)) * 100
high_missing = missing_pct[missing_pct > 50].index.tolist()
print(f"Columns with >50% missing values: {high_missing}")

# Drop columns with high missing percentages and the Id column
columns_to_drop = ['Id'] + high_missing
datan = data.drop(columns=columns_to_drop, errors='ignore').copy()

# Handle missing values in remaining columns
# For numerical columns: impute with median
numeric_cols_with_missing = datan.select_dtypes(include=[np.number]).columns[datan.select_dtypes(include=[np.number]).isnull().any()].tolist()
for col in numeric_cols_with_missing:
    datan.loc[:, col] = datan[col].fillna(datan[col].median())

# For categorical columns: impute with mode or 'None' for meaningful features
categorical_cols_with_missing = datan.select_dtypes(include=['object']).columns[datan.select_dtypes(include=['object']).isnull().any()].tolist()
for col in categorical_cols_with_missing:
    # For features where 'None' is meaningful (e.g., no garage, no basement)
    if col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
               'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
               'FireplaceQu', 'Fence', 'MiscFeature']:
        datan.loc[:, col] = datan[col].fillna('None')
    else:
        datan.loc[:, col] = datan[col].fillna(datan[col].mode()[0])

print(f"\nShape after cleaning: {datan.shape}")
print(f"Remaining missing values: {datan.isnull().sum().sum()}")
datan.head()

Columns with >50% missing values: ['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']

Shape after cleaning: (1460, 76)
Remaining missing values: 0


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SalePrice_log
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2,2008,WD,Normal,208500,12.247699
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,5,2007,WD,Normal,181500,12.109016
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,9,2008,WD,Normal,223500,12.317171
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,2,2006,WD,Abnorml,140000,11.849405
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,12,2008,WD,Normal,250000,12.429220


In [69]:
# 3. Outliers Box Plot Visualization
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
import numpy as np

# Use datan if available, otherwise use data
df_to_use = datan if 'datan' in globals() else data

# Select top features with outliers for visualization
top_outlier_features = ['SalePrice', 'GrLivArea', 'LotArea', 'TotalBsmtSF', '1stFlrSF']

# Prepare data for box plot
box_data = []
for feature in top_outlier_features:
    if feature in df_to_use.columns:
        values = df_to_use[feature].dropna()
        q1 = values.quantile(0.25)
        q2 = values.quantile(0.50)
        q3 = values.quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr

        # Identify outliers
        outliers = values[(values < lower) | (values > upper)]

        box_data.append({
            'feature': feature,
            'q1': q1,
            'median': q2,
            'q3': q3,
            'lower': lower,
            'upper': upper,
            'outlier_count': len(outliers)
        })

# Create box plot using numeric positions
feature_names = [d['feature'] for d in box_data]
x_positions = list(range(len(box_data)))

p = figure(width=800, height=500, title="Outliers Box Plot - Top Features",
           x_range=feature_names,
           x_axis_label="Features", y_axis_label="Values",
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

# Draw boxes
for i, d in enumerate(box_data):
    x = i  # Use numeric position
    # Box
    p.rect(x=x, y=(d['q1'] + d['q3'])/2, width=0.6, height=d['q3'] - d['q1'],
           fill_color='lightblue', line_color='navy', alpha=0.7)
    # Median line
    p.line([x-0.3, x+0.3], [d['median'], d['median']], line_width=2, color='red')
    # Whiskers
    p.line([x, x], [d['lower'], d['q1']], line_width=2, color='navy')
    p.line([x, x], [d['q3'], d['upper']], line_width=2, color='navy')
    p.line([x-0.2, x+0.2], [d['lower'], d['lower']], line_width=2, color='navy')
    p.line([x-0.2, x+0.2], [d['upper'], d['upper']], line_width=2, color='navy')

p.xaxis.major_label_orientation = 1.2
show(p)


In [70]:
# OUTLIER DETECTION
numeric_col = datan.select_dtypes(include=['int64', 'float64']).columns
outlier_indices = {}

for col in numeric_col:
    Q1 = datan[col].quantile(0.25)
    Q3 = datan[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = datan[(datan[col] < lower_bound) | (datan[col] > upper_bound)]
    outlier_indices[col] = outliers.index.tolist()

    if len(outliers) > 0:
        print(f"{col}: {len(outliers)} outliers")

print(f"\nTotal columns checked: {len(numeric_col)}")

# --- Outlier Inspection ---
top_corr = corr[target].abs().sort_values(ascending=False).head(6).index
top_corr_list = list(top_corr)

# Create pairplot using Bokeh
from itertools import combinations
plots = []
for i, col1 in enumerate(top_corr_list):
    row_plots = []
    for j, col2 in enumerate(top_corr_list):
        if i == j:
            # Diagonal: histogram
            col_data = data[col1].dropna()
            hist, edges = np.histogram(col_data, bins=20)
            p = figure(width=400, height=250, toolbar_location=None)
            p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
                   fill_color='steelblue', line_color='white', alpha=0.7)
            row_plots.append(p)
        else:
            # Off-diagonal: scatter plot
            p = figure(width=400, height=250, toolbar_location=None)
            p.circle(data[col2], data[col1], size=2, alpha=0.5, color='steelblue')
            row_plots.append(p)
    plots.append(row_plots)

grid = gridplot(plots, toolbar_location='above')
show(grid)

MSSubClass: 103 outliers
LotFrontage: 106 outliers
LotArea: 69 outliers
OverallQual: 2 outliers
OverallCond: 125 outliers
YearBuilt: 7 outliers
MasVnrArea: 98 outliers
BsmtFinSF1: 7 outliers
BsmtFinSF2: 167 outliers
BsmtUnfSF: 29 outliers
TotalBsmtSF: 61 outliers
1stFlrSF: 20 outliers
2ndFlrSF: 2 outliers
LowQualFinSF: 26 outliers
GrLivArea: 31 outliers
BsmtFullBath: 1 outliers
BsmtHalfBath: 82 outliers
BedroomAbvGr: 35 outliers
KitchenAbvGr: 68 outliers
TotRmsAbvGrd: 30 outliers
Fireplaces: 5 outliers
GarageYrBlt: 1 outliers
GarageCars: 5 outliers
GarageArea: 21 outliers
WoodDeckSF: 32 outliers
OpenPorchSF: 77 outliers
EnclosedPorch: 208 outliers
3SsnPorch: 24 outliers
ScreenPorch: 116 outliers
PoolArea: 7 outliers
MiscVal: 52 outliers
SalePrice: 61 outliers
SalePrice_log: 28 outliers

Total columns checked: 38


In [71]:
# ==========================================
# 🧮 Outlier Capping / Winsorization
# ==========================================
from scipy.stats.mstats import winsorize

# Identify numeric columns
numeric_cols = datan.select_dtypes(include=[np.number]).columns

# Apply Winsorization (limit at 1st–99th percentile)
for col in numeric_cols:
    lower, upper = datan[col].quantile([0.01, 0.99])
    datan[col] = np.clip(datan[col], lower, upper)

print("✅ Outliers capped at 1st–99th percentile for numeric columns.")


✅ Outliers capped at 1st–99th percentile for numeric columns.


# Note: Outliers found but not removing them as they may be legitimate high-value properties
# For house price prediction, extreme values can be real luxury homes

# **FEATURE ENGINEERING**

In [72]:
# CREATE NEW FEATURES FROM EXISTING COLUMNS

# 1. Total Square Footage
datan['TotalSF'] = datan['TotalBsmtSF'] + datan['1stFlrSF'] + datan['2ndFlrSF']

# 2. Property Age (from year sold)
datan['PropertyAge'] = datan['YrSold'] - datan['YearBuilt']

# 3. Years Since Remodel
datan['RemodAge'] = datan['YrSold'] - datan['YearRemodAdd']

# 4. Binary features - Has Garage
datan['HasGarage'] = (datan['GarageArea'] > 0).astype(int)

# 5. Binary features - Has Basement
datan['HasBasement'] = (datan['TotalBsmtSF'] > 0).astype(int)

# 6. Binary features - Has Fireplace
datan['HasFireplace'] = (datan['Fireplaces'] > 0).astype(int)

# 7. Binary features - Has 2nd Floor
datan['Has2ndFloor'] = (datan['2ndFlrSF'] > 0).astype(int)

# 8. Quality Score (interaction feature)
datan['QualityScore'] = datan['OverallQual'] * datan['OverallCond']

# 9. Total Bathrooms
datan['TotalBath'] = datan['FullBath'] + 0.5 * datan['HalfBath'] + datan['BsmtFullBath'] + 0.5 * datan['BsmtHalfBath']

# 10. Total Porch Area
datan['TotalPorchSF'] = datan['OpenPorchSF'] + datan['EnclosedPorch'] + datan['3SsnPorch'] + datan['ScreenPorch']

print("New features created:")
print("TotalSF, PropertyAge, RemodAge, HasGarage, HasBasement, HasFireplace,")
print("Has2ndFloor, QualityScore, TotalBath, TotalPorchSF")
print(f"\nUpdated shape: {datan.shape}")
datan[['TotalSF', 'PropertyAge', 'RemodAge', 'HasGarage', 'HasBasement', 'QualityScore', 'TotalBath']].head()

New features created:
TotalSF, PropertyAge, RemodAge, HasGarage, HasBasement, HasFireplace,
Has2ndFloor, QualityScore, TotalBath, TotalPorchSF

Updated shape: (1460, 86)


,TotalSF,PropertyAge,RemodAge,HasGarage,HasBasement,QualityScore,TotalBath
0,2566.0,5.0,5,1,1,35,3.5
1,2524.0,31.0,31,1,1,48,2.5
2,2706.0,7.0,6,1,1,35,3.5
3,2473.0,91.0,36,1,1,35,2.0
4,3343.0,8.0,8,1,1,40,3.5


In [73]:
# One-hot encode categorical variables
# Select only categorical columns
categorical_cols = datan.select_dtypes(include=['object']).columns.tolist()

# Limit to important categorical features to avoid too many dummy variables
important_categoricals = ['Neighborhood', 'BldgType', 'HouseStyle', 'ExterQual', 'ExterCond',
                          'Foundation', 'HeatingQC', 'CentralAir', 'KitchenQual',
                          'GarageType', 'GarageFinish', 'PavedDrive', 'SaleCondition']

# Keep only those that exist in our data
categorical_to_encode = [col for col in important_categoricals if col in categorical_cols]

# Create dummy variables
datam = pd.get_dummies(datan, columns=categorical_to_encode, drop_first=True)

print(f"Shape after one-hot encoding: {datam.shape}")
print(f"Encoded categorical columns: {categorical_to_encode}")
datam.head()

Shape after one-hot encoding: (1460, 144)
Encoded categorical columns: ['Neighborhood', 'BldgType', 'HouseStyle', 'ExterQual', 'ExterCond', 'Foundation', 'HeatingQC', 'CentralAir', 'KitchenQual', 'GarageType', 'GarageFinish', 'PavedDrive', 'SaleCondition']


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,False,True,False,False,True,False,False,False,True,False
1,20,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,False,True,False,False,True,False,False,False,True,False
2,60,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,False,True,False,False,True,False,False,False,True,False
3,70,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,False,False,True,False,True,False,False,False,False,False
4,60,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,False,True,False,False,True,False,False,False,True,False


In [74]:
datam.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,False,True,False,False,True,False,False,False,True,False
1,20,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,False,True,False,False,True,False,False,False,True,False
2,60,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,False,True,False,False,True,False,False,False,True,False
3,70,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,False,False,True,False,True,False,False,False,False,False
4,60,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,False,True,False,False,True,False,False,False,True,False


In [75]:
# Using LabelEncoder to transform remaining categorical variables to numeric variables

from sklearn.preprocessing import LabelEncoder

# Separate features and target
X = datam.drop(columns=['SalePrice', 'SalePrice_log'], axis=1)
y = datam['SalePrice']

# Find remaining categorical columns (those not one-hot encoded)
cat_f = X.select_dtypes(include='object').columns
print(f"Remaining categorical columns to encode: {list(cat_f)}")

# Encode remaining categorical features
X_encoded = X.copy()
label_encoders = {}
for col in cat_f:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col])
    label_encoders[col] = le

print(f"\nFinal feature shape: {X_encoded.shape}")
print(f"Target variable shape: {y.shape}")
X_encoded.head()

Remaining categorical columns to encode: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Electrical', 'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'SaleType']

Final feature shape: (1460, 142)
Target variable shape: (1460,)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,3,65.0,8450.0,1,3,3,0,4,0,...,False,True,False,False,True,False,False,False,True,False
1,20,3,80.0,9600.0,1,3,3,0,2,0,...,False,True,False,False,True,False,False,False,True,False
2,60,3,68.0,11250.0,1,0,3,0,4,0,...,False,True,False,False,True,False,False,False,True,False
3,70,3,60.0,9550.0,1,0,3,0,0,0,...,False,False,True,False,True,False,False,False,False,False
4,60,3,84.0,14260.0,1,0,3,0,2,0,...,False,True,False,False,True,False,False,False,True,False


In [76]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

# Calculate mutual information scores for all features
selector = SelectKBest(score_func=mutual_info_regression, k='all')
selector.fit(X_encoded, y)

mi_scores = pd.DataFrame({
    'Feature': X_encoded.columns,
    'MI Score': selector.scores_
}).sort_values(by='MI Score', ascending=False)

print("Top 20 Features by Mutual Information Score:")
print(mi_scores.head(20))

Top 20 Features by Mutual Information Score:
          Feature  MI Score
61        TotalSF  0.679343
12    OverallQual  0.582810
35      GrLivArea  0.479703
68   QualityScore  0.450463
14      YearBuilt  0.376487
29    TotalBsmtSF  0.371827
48     GarageArea  0.366145
47     GarageCars  0.363413
69      TotalBath  0.357328
62    PropertyAge  0.352393
21       BsmtQual  0.328128
32       1stFlrSF  0.311081
46    GarageYrBlt  0.305605
0      MSSubClass  0.281255
108  ExterQual_TA  0.271971
63       RemodAge  0.260318
38       FullBath  0.242963
15   YearRemodAdd  0.242800
107  ExterQual_Gd  0.226484
2     LotFrontage  0.217729


In [77]:
from sklearn.feature_selection import f_regression

# Apply f_regression
f_selector = SelectKBest(score_func=f_regression, k='all')
f_selector.fit(X_encoded, y)

# Collect scores
f_scores = pd.DataFrame({
    'Feature': X_encoded.columns,
    'F_Score': f_selector.scores_,
    'p_value': f_selector.pvalues_
}).sort_values(by='F_Score', ascending=False)

print("Top 20 Features by F-Score (Linear Relation with SalePrice):")
print(f_scores.head(20))

Top 20 Features by F-Score (Linear Relation with SalePrice):
              Feature      F_Score        p_value
61            TotalSF  3016.894195   0.000000e+00
12        OverallQual  2781.015721   0.000000e+00
35          GrLivArea  1584.494612  3.681976e-235
47         GarageCars  1160.316191  1.372357e-187
48         GarageArea  1058.084440  5.731072e-175
69          TotalBath  1051.548353  3.824058e-174
29        TotalBsmtSF  1049.422880  7.096414e-174
32           1stFlrSF   948.790642  6.797806e-161
108      ExterQual_TA   874.930548  5.143659e-151
21           BsmtQual   845.895488  4.796464e-147
68       QualityScore   733.883653  3.021845e-131
38           FullBath   733.784067  3.123751e-131
62        PropertyAge   638.905536  3.354464e-117
14          YearBuilt   637.335379  5.796311e-117
125    KitchenQual_TA   604.334852  6.269094e-112
42       TotRmsAbvGrd   598.819427  4.430431e-111
63           RemodAge   573.647144  3.561830e-107
15       YearRemodAdd   567.573000  3.1

In [78]:
# Train Random Forest model to get feature importance
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_encoded, y)

# Get feature importance
importances = pd.Series(rf.feature_importances_, index=X_encoded.columns)
top_features = importances.sort_values(ascending=False).head(30)

print("Top 20 Features by Random Forest Importance:")
print(top_features)

# Visualize top features with Bokeh
top_features_df = top_features.reset_index()
top_features_df.columns = ['Feature', 'Importance']
# Ensure Feature column is string type
top_features_df['Feature'] = top_features_df['Feature'].astype(str)
# Sort by Importance for proper display (ascending for bottom-to-top)
top_features_df = top_features_df.sort_values('Importance', ascending=True)

# Create y_range from sorted features
y_range_list = top_features_df['Feature'].tolist()

p = figure(width=700, height=600, title='Top 30 Most Important Features',
           y_range=y_range_list,
           x_axis_label='Feature Importance', y_axis_label='Feature',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source = ColumnDataSource(top_features_df)
p.hbar(y='Feature', right='Importance', source=source, height=0.8,
       fill_color='steelblue', line_color='white')

show(p)

Top 20 Features by Random Forest Importance:
TotalSF                 0.464340
OverallQual             0.318256
QualityScore            0.026020
YearBuilt               0.018375
LotArea                 0.012195
PropertyAge             0.011211
BsmtFinSF1              0.009091
TotalBath               0.008743
GrLivArea               0.008433
GarageArea              0.008406
GarageCars              0.008218
BsmtQual                0.007922
BsmtUnfSF               0.006738
LotFrontage             0.005741
RemodAge                0.005634
YearRemodAdd            0.004894
LandContour             0.003460
2ndFlrSF                0.003306
1stFlrSF                0.003227
TotalPorchSF            0.002974
OpenPorchSF             0.002949
GarageYrBlt             0.002940
TotalBsmtSF             0.002858
MoSold                  0.002836
KitchenQual_Gd          0.002620
Neighborhood_Edwards    0.002509
WoodDeckSF              0.002467
MasVnrArea              0.002382
OverallCond             0.00187

In [79]:
from sklearn.feature_selection import f_regression, mutual_info_regression
import pandas as pd

# --- Target choices ---
targets = {
    "SalePrice": datan["SalePrice"],
    "SalePrice_log": datan["SalePrice_log"]
}

# --- Loop through both targets ---
results = {}

for name, y in targets.items():
    f_selector = SelectKBest(score_func=f_regression, k='all')
    f_selector.fit(X_encoded, y)

    f_scores = pd.DataFrame({
        'Feature': X_encoded.columns,
        'F_Score': f_selector.scores_,
        'p_value': f_selector.pvalues_
    }).sort_values(by='F_Score', ascending=False)

    mi_scores = pd.Series(mutual_info_regression(X_encoded, y, random_state=42),
                          index=X_encoded.columns, name='MI_Score').sort_values(ascending=False)

    results[name] = (f_scores, mi_scores)

# --- Compare top 10 for both targets ---
print("Top F-test features for SalePrice_log:")
display(results["SalePrice_log"][0].head(10))

print("Top Mutual Information features for SalePrice_log:")
display(results["SalePrice_log"][1].head(10))

print("Top F-test features for SalePrice (raw):")
display(results["SalePrice"][0].head(10))


Top F-test features for SalePrice_log:


,Feature,F_Score,p_value
61,TotalSF,2982.628326,0.000000e+00
12,OverallQual,2934.708792,0.000000e+00
35,GrLivArea,1556.208807,3.350700e-232
47,GarageCars,1324.044421,8.293271e-207
69,TotalBath,1233.779067,2.341753e-196
48,GarageArea,1144.896414,1.021222e-185
29,TotalBsmtSF,1040.129188,1.066135e-172
32,1stFlrSF,888.106146,8.437895e-153
108,ExterQual_TA,853.851351,3.874651e-148
38,FullBath,841.701618,1.813209e-146


Top Mutual Information features for SalePrice_log:


TotalSF         0.683332
OverallQual     0.571558
GrLivArea       0.463590
QualityScore    0.453042
GarageArea      0.371915
YearBuilt       0.370918
TotalBsmtSF     0.366724
GarageCars      0.365364
TotalBath       0.343462
PropertyAge     0.330852
Name: MI_Score, dtype: float64

Top F-test features for SalePrice (raw):


,Feature,F_Score,p_value
61,TotalSF,3016.894195,0.000000e+00
12,OverallQual,2781.015721,0.000000e+00
35,GrLivArea,1584.494612,3.681976e-235
47,GarageCars,1160.316191,1.372357e-187
48,GarageArea,1058.084440,5.731072e-175
69,TotalBath,1051.548353,3.824058e-174
29,TotalBsmtSF,1049.422880,7.096414e-174
32,1stFlrSF,948.790642,6.797806e-161
108,ExterQual_TA,874.930548,5.143659e-151
21,BsmtQual,845.895488,4.796464e-147


In [80]:
# --- Combine F-test and Mutual Information top 30 ---
top_f = f_scores.head(30)['Feature'].tolist()
top_mi = mi_scores.head(30).index.tolist()

# Combine both sets (union)
filter_top30 = list(set(top_f) | set(top_mi))
print(f"Total unique top features from filters: {len(filter_top30)}")

X_filter = X_encoded[filter_top30]
y_log = datam['SalePrice_log']

Total unique top features from filters: 35


In [81]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_absolute_error

# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_filter,  # from filter-based selection (top 30)
    datan["SalePrice_log"],  # target (use log version)
    test_size=0.2,
    random_state=42
)

# --- Random Forest Regressor ---
rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

# --- Cross-validation setup ---
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# --- RFECV setup ---
rfecv = RFECV(
    estimator=rf,
    step=1,         # remove one feature per iteration
    cv=cv,
    scoring="r2",   # use R² for scoring
    n_jobs=-1
)

# --- Fit the selector ---
rfecv.fit(X_train, y_train)

# --- Get selected features ---
selected_rfecv = X_filter.columns[rfecv.support_].tolist()
print(f"[RFECV] Optimal #features: {rfecv.n_features_}")
print("Selected features:", selected_rfecv)

# --- Evaluate performance on hold-out set ---
rf.fit(X_train[selected_rfecv], y_train)
pred = rf.predict(X_test[selected_rfecv])

r2 = r2_score(y_test, pred)
mae = mean_absolute_error(y_test, pred)
print(f"[RFECV] Hold-out R²: {r2:.4f} | MAE: {mae:.3f}")

# Plot RFECV performance with Bokeh
n_features = list(range(1, len(rfecv.cv_results_['mean_test_score']) + 1))
scores = rfecv.cv_results_['mean_test_score']

p = figure(width=700, height=400, title="RFECV Feature Selection Performance",
           x_axis_label="Number of Features Selected",
           y_axis_label="Cross-Validated R²",
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

p.line(n_features, scores, line_width=2, color='steelblue', legend_label='R² Score')
p.circle(n_features, scores, size=5, color='steelblue')

show(p)

[RFECV] Optimal #features: 24
Selected features: ['TotalSF', 'BsmtFinSF1', 'TotalBath', 'PropertyAge', 'LotFrontage', 'Fireplaces', 'GarageCars', '2ndFlrSF', 'TotalBsmtSF', '1stFlrSF', 'LotArea', 'MSSubClass', 'KitchenQual_TA', 'OverallQual', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'RemodAge', 'GarageType_Detchd', 'Exterior2nd', 'YearRemodAdd', 'YearBuilt', 'QualityScore', 'BsmtQual']
[RFECV] Hold-out R²: 0.8972 | MAE: 0.092


In [82]:
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error

# --- Train-test split (reuse previous or recreate) ---
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_filter,              # top features from filter stage
    datan["SalePrice_log"],  # target
    test_size=0.2,
    random_state=42
)

# --- Build pipeline: scaling + Lasso with cross-validation ---
lasso_pipe = make_pipeline(
    StandardScaler(),
    LassoCV(cv=5, random_state=42, n_jobs=-1)
)

# --- Fit model ---
lasso_pipe.fit(X_train, y_train)

# --- Retrieve coefficients ---
lasso = lasso_pipe.named_steps["lassocv"]
coef_series = pd.Series(lasso.coef_, index=X_filter.columns)

# --- Non-zero coefficients (selected features) ---
lasso_selected = coef_series[coef_series != 0].sort_values(key=abs, ascending=False)
print(f"[LASSO] Selected {len(lasso_selected)} features")
display(lasso_selected.head(15))

# --- Evaluate performance ---
y_pred = lasso_pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"[LASSO] Hold-out R²: {r2:.4f} | MAE: {mae:.4f}")

# Plot Lasso features with Bokeh
lasso_top = lasso_selected.head(30).abs().reset_index()
lasso_top.columns = ['Feature', 'Coefficient']
lasso_top = lasso_top.sort_values('Coefficient')

p = figure(width=700, height=600, title="Top Lasso-Selected Features (Coefficient Magnitude)",
           y_range=lasso_top['Feature'].tolist(),
           x_axis_label="Coefficient (Absolute Value)", y_axis_label='Feature',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source = ColumnDataSource(lasso_top)
p.hbar(y='Feature', right='Coefficient', source=source, height=0.8,
       fill_color='coral', line_color='white')

show(p)



[LASSO] Selected 23 features


GrLivArea           0.118009
QualityScore        0.083249
PropertyAge        -0.073388
TotalBsmtSF         0.041482
GarageCars          0.035695
BsmtFinSF1          0.031033
Fireplaces          0.029663
Foundation_PConc    0.025242
TotalBath           0.020507
LotArea             0.019861
MSSubClass         -0.019357
RemodAge           -0.016624
BsmtQual           -0.014929
OverallQual         0.013903
GarageArea          0.012173
dtype: float64

[LASSO] Hold-out R²: 0.8977 | MAE: 0.0973


In [83]:
# --- Combine all unique features ---
# Convert all to sets for proper set operations
filter_set = set(filter_top30)
rfecv_set = set(selected_rfecv)
lasso_set = set(lasso_selected.index.tolist())  # Convert Index to list first, then to set

combined_features = filter_set | rfecv_set | lasso_set
common_features = filter_set & rfecv_set & lasso_set

print(f"Total unique selected features: {len(combined_features)}")
print(f"Highly consistent across all methods: {len(common_features)}")

print("\nCommon features across ALL methods:")
print(sorted(list(common_features)))


Total unique selected features: 35
Highly consistent across all methods: 15

Common features across ALL methods:
['BsmtFinSF1', 'BsmtQual', 'Fireplaces', 'GarageArea', 'GarageCars', 'GrLivArea', 'KitchenQual_TA', 'LotArea', 'MSSubClass', 'OverallQual', 'PropertyAge', 'QualityScore', 'RemodAge', 'TotalBath', 'TotalBsmtSF']


In [84]:
X_final = X_encoded[list(combined_features)]
y_final = datan["SalePrice_log"]

print("Final dataset shape:", X_final.shape)


Final dataset shape: (1460, 35)


In [85]:
# 2. PCA Plot - First Two Principal Components
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
import numpy as np

# Use the PCA transformed data (X_reduced or X_pca)
# If X_reduced exists, use first 2 components; otherwise use X_pca
if 'X_reduced' in globals():
    pca_data = X_reduced[:, :2]  # First 2 components
elif 'X_pca' in globals():
    pca_data = X_pca[:, :2]
else:
    # Recompute if needed
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_final)
    pca_temp = PCA(n_components=2, random_state=42)
    pca_data = pca_temp.fit_transform(X_scaled)

# Create scatter plot
source = ColumnDataSource(data={
    'PC1': pca_data[:, 0],
    'PC2': pca_data[:, 1],
    'SalePrice': datan['SalePrice'].values if 'datan' in globals() else data['SalePrice'].values[:len(pca_data)]
})

p = figure(width=700, height=500, title="PCA Plot - First Two Principal Components",
           x_axis_label="First Principal Component (PC1)",
           y_axis_label="Second Principal Component (PC2)",
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

# Add hover tool
hover = HoverTool(tooltips=[("PC1", "@PC1{0.2f}"), ("PC2", "@PC2{0.2f}"), ("SalePrice", "@SalePrice{0,0}")])
p.add_tools(hover)

# Color by SalePrice
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256

mapper = linear_cmap(field_name='SalePrice', palette=Viridis256,
                      low=min(source.data['SalePrice']), high=max(source.data['SalePrice']))

p.circle('PC1', 'PC2', source=source, size=5, alpha=0.6, color=mapper, line_color='white', line_width=0.5)

show(p)


In [86]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# --- Scale the data ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

# --- Apply PCA ---
pca = PCA(n_components=None, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# --- Explained variance ratio ---
explained_var = np.cumsum(pca.explained_variance_ratio_)

# --- Plot how much variance is explained with Bokeh ---
n_components = list(range(1, len(explained_var)+1))

p = figure(width=700, height=400, title='PCA Variance Explained by Components',
           x_axis_label='Number of Principal Components',
           y_axis_label='Cumulative Explained Variance',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

p.line(n_components, explained_var, line_width=2, color='steelblue', legend_label='Cumulative Variance')
p.circle(n_components, explained_var, size=6, color='steelblue')

show(p)


In [87]:
n_components_95 = np.argmax(explained_var >= 0.95) + 1
print(f"Number of components to retain 95% variance: {n_components_95}")
# Reducing data
pca_final = PCA(n_components=n_components_95, random_state=42)
X_reduced = pca_final.fit_transform(X_scaled)

print("Reduced shape:", X_reduced.shape)


Number of components to retain 95% variance: 20
Reduced shape: (1460, 20)


In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y_final, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)

r2 = r2_score(y_test, pred)
mae = mean_absolute_error(y_test, pred)
print(f"[PCA Reduced Model] R²: {r2:.4f} | MAE: {mae:.4f}")


[PCA Reduced Model] R²: 0.8992 | MAE: 0.0930


# **Model Training**

### **Gradient Boosting**

In [89]:
# Gradient Boosting Regressor (PCA)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import joblib

gb_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.8,
    random_state=42
)

gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

# Save
joblib.dump(gb_model, "model_gb_pca.pkl")
print("Gradient Boosting trained and saved as model_gb_pca.pkl")

# Keep predictions & true for final comparison
gb_pred_log = y_pred_gb


Gradient Boosting trained and saved as model_gb_pca.pkl


### **SVR**

In [90]:
# Tuned SVR (PCA) using Pipeline + GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import joblib
import time

svr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR())
])

svr_param_grid = {
    "svr__kernel": ["rbf"],
    "svr__C": [1.0, 10.0],
    "svr__epsilon": [0.05, 0.1],
    "svr__gamma": ["scale"]
}

print("Starting SVR GridSearchCV (may take some time)...")
t0 = time.time()
svr_gs = GridSearchCV(svr_pipe, svr_param_grid, cv=3, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)
svr_gs.fit(X_train, y_train)
t1 = time.time()
print(f"SVR GridSearch finished in {t1-t0:.1f}s")
print("Best SVR params:", svr_gs.best_params_)

svr_best = svr_gs.best_estimator_
y_pred_svr = svr_best.predict(X_test)

# Save
joblib.dump(svr_best, "model_svr_pca.pkl")
print("Tuned SVR trained and saved as model_svr_pca.pkl")

# Keep predictions & true for final comparison
svr_pred_log = y_pred_svr


Starting SVR GridSearchCV (may take some time)...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
SVR GridSearch finished in 1.1s
Best SVR params: {'svr__C': 1.0, 'svr__epsilon': 0.05, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
Tuned SVR trained and saved as model_svr_pca.pkl


### **Light GBM**

In [91]:
# LightGBM Regressor (PCA) — with graceful fallback if LightGBM not installed
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, r2_score

try:
    import lightgbm as lgb

    lgb_model = lgb.LGBMRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )

    lgb_model.fit(X_train, y_train)
    y_pred_lgb = lgb_model.predict(X_test)
    joblib.dump(lgb_model, "model_lgb_pca.pkl")
    print("LightGBM trained and saved as model_lgb_pca.pkl")

except Exception as e:
    # If LightGBM cannot be used, fallback to GradientBoosting predictions (or create NaN preds)
    print("LightGBM is not available or failed to import/fit. Falling back to GradientBoosting predictions.")
    try:
        y_pred_lgb = gb_pred_log.copy()   # reuse GB predictions if available
        print("Using GradientBoosting predictions as LightGBM fallback.")
    except NameError:
        # create same-shape copy of y_test as fallback (so final comparison still runs)
        y_pred_lgb = y_test.copy()
        print("No GB predictions available; using y_test as placeholder for LightGBM predictions.")

# Keep predictions & true for final comparison
lgb_pred_log = y_pred_lgb


LightGBM trained and saved as model_lgb_pca.pkl


### **Linear Regression**

In [92]:
# Linear Regression (PCA)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import joblib

# Train
lr_pca = LinearRegression()
lr_pca.fit(X_train, y_train)

# Predict (log-target)
y_pred_lr_pca = lr_pca.predict(X_test)

# Save
joblib.dump(lr_pca, "model_lr_pca.pkl")
print("Linear Regression trained and saved as model_lr_pca.pkl")

# Keep predictions & true for final comparison
lr_pred_log = y_pred_lr_pca


Linear Regression trained and saved as model_lr_pca.pkl


# **RMSE (log-target) and R² for all 4 models**

In [93]:
# Final comparison table (RMSE and R² on log-target)
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Ensure all prediction variables exist; if not, create placeholders to avoid errors
preds = {}
preds['LinearRegression'] = globals().get('lr_pred_log', None)
preds['GradientBoosting'] = globals().get('gb_pred_log', globals().get('y_pred_gb', None))
preds['TunedSVR'] = globals().get('svr_pred_log', globals().get('y_pred_svr', None))
preds['LightGBM'] = globals().get('lgb_pred_log', globals().get('y_pred_lgb', None))

results = []
for name, y_pred in preds.items():
    if y_pred is None:
        # missing prediction -> skip
        results.append({'Model': name, 'RMSE_log': np.nan, 'R2_log': np.nan})
        continue
    # flatten in case
    y_true = np.ravel(y_test)
    y_pred_arr = np.ravel(y_pred)
    mse = mean_squared_error(y_true, y_pred_arr)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred_arr)
    results.append({'Model': name, 'RMSE_log': rmse, 'R2_log': r2})

df_results = pd.DataFrame(results).sort_values(by='RMSE_log')
# Pretty print: RMSE as 4-decimals (log-target scale), R2 4-decimals
print("\nModel comparison (metrics computed on log-target; no inverse transform used):")
print(df_results.to_string(index=False, formatters={'RMSE_log':'{:.4f}'.format, 'R2_log':'{:.4f}'.format}))

# Also show as a nicer table
df_results_display = df_results.copy()
df_results_display['RMSE_log'] = df_results_display['RMSE_log'].map(lambda x: np.nan if np.isnan(x) else float(f"{x:.4f}"))
df_results_display['R2_log'] = df_results_display['R2_log'].map(lambda x: np.nan if np.isnan(x) else float(f"{x:.4f}"))
df_results_display = df_results_display.reset_index(drop=True)
df_results_display



Model comparison (metrics computed on log-target; no inverse transform used):
           Model RMSE_log R2_log
GradientBoosting   0.1267 0.9043
        LightGBM   0.1295 0.9001
LinearRegression   0.1333 0.8942
        TunedSVR   0.1419 0.8801


,Model,RMSE_log,R2_log
0,GradientBoosting,0.1267,0.9043
1,LightGBM,0.1295,0.9001
2,LinearRegression,0.1333,0.8942
3,TunedSVR,0.1419,0.8801


In [94]:
# **FINAL OUTCOME VISUALIZATIONS - Telling the Story**

# Visualization 1: Model Performance Comparison (R² Scores)
# Sort by R² for better visualization
df_viz = df_results.sort_values('R2_log', ascending=True).copy()
df_viz['Color'] = ['#2ecc71' if model == 'GradientBoosting' else '#3498db' for model in df_viz['Model']]

p1 = figure(width=700, height=400, title='Model Performance Comparison - R² Scores',
           y_range=df_viz['Model'].tolist(),
           x_axis_label='R² Score', y_axis_label='Model',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source1 = ColumnDataSource(df_viz)
p1.hbar(y='Model', right='R2_log', source=source1, height=0.6,
        fill_color='Color', line_color='white', line_width=1.5)

# Add hover tool
hover1 = HoverTool(tooltips=[("Model", "@Model"), ("R² Score", "@R2_log{0.0000}"), ("RMSE", "@RMSE_log{0.0000}")])
p1.add_tools(hover1)

# Visualization 2: Best Model Prediction Accuracy (GradientBoosting)
# Convert log predictions back to original scale for better interpretation
y_test_original = np.expm1(y_test)
y_pred_original = np.expm1(gb_pred_log)

# Create scatter plot
p2 = figure(width=700, height=600, title='Best Model Prediction Accuracy - GradientBoosting',
           x_axis_label='Actual Sale Price (log scale)', y_axis_label='Predicted Sale Price (log scale)',
           toolbar_location='above', tools='hover,pan,box_zoom,reset,save')

source2 = ColumnDataSource({
    'actual': y_test,
    'predicted': gb_pred_log,
    'actual_orig': y_test_original,
    'predicted_orig': y_pred_original
})

p2.scatter('actual', 'predicted', source=source2, size=4, alpha=0.6, color='steelblue')

# Add perfect prediction line (y=x)
min_val = min(min(y_test), min(gb_pred_log))
max_val = max(max(y_test), max(gb_pred_log))
p2.line([min_val, max_val], [min_val, max_val], line_width=2, line_color='red',
        line_dash='dashed', legend_label='Perfect Prediction')

# Add hover tool
hover2 = HoverTool(tooltips=[("Actual (log)", "@actual{0.0000}"),
                              ("Predicted (log)", "@predicted{0.0000}"),
                              ("Actual ($)", "@actual_orig{$0,0}"),
                              ("Predicted ($)", "@predicted_orig{$0,0}")])
p2.add_tools(hover2)
p2.legend.location = 'top_left'

# Display visualizations
show(column(p1, p2))
